In [1]:
import random
import weave
weave.use_frontend_devmode()
from rich import print
from weave.ecosystem import scenario

In [2]:
def make_run(n_scenarios) -> list[scenario.ScenarioResult]:
    return [
        scenario.ScenarioResult(
            scenario_id=str(i),
            metric1=random.random(),
            metric2=random.random(),
            metric3=random.random())
        for i in range(n_scenarios)
    ]

In [3]:
panel_input = weave.save({'baseline': make_run(10), 'candidate': make_run(10)}, name='panel_input')

In [4]:
panel_input